In [ ]:
import os
import os.path as path
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import math
import imageio
import glob
from skimage import io
import albumentations as A
import json
import random
from numpy import savez_compressed
import shutil
from random import choice

In [ ]:
OutputFolder = "../dataset/"  ##Argument 1
directory = "finalDataset" 
path = os.path.join(OutputFolder, directory)
os.mkdir(path)

In [ ]:
Augmented_json_file = "../dataset/augmentations/Level0/augmentedJSON.json"  ##Argument 2
ImageFolderPath = "../dataset/augmentations/Level0/"  ##Argument 3
keypoints = True      #Make sure the keypoints label in JSON is kp-1  ##Argument 4
OutputFolder = path + "/"

Run code without any change

In [ ]:
directory = "mix"
path = os.path.join(OutputFolder, directory)
os.mkdir(path)

In [ ]:
path_tostoreImagesAndAnnotations = path +"/"

In [ ]:
# fetch all files
for file_name in os.listdir(ImageFolderPath):
    if (os.path.splitext(file_name)[1] == ".jpg") or (os.path.splitext(file_name)[1] == ".png") or (os.path.splitext(file_name)[1] == ".jpeg"):
        source = ImageFolderPath + file_name
        destination = path_tostoreImagesAndAnnotations + file_name
        if os.path.isfile(source):
            shutil.copy(source, destination)
            os.remove(source)

In [ ]:
with open(Augmented_json_file) as f:
    imgs_anns = json.load(f)

classNames = []
listOfImages=[]
for Count,val in enumerate(imgs_anns):
    fileName = val["image"]
    ImgId = val["id"]
    if "kp-1" in val:        
        for CountLabels,valLabels in enumerate(val["kp-1"]):
            if len(classNames) == 0:
                classNames.append(valLabels["keypointlabels"][0])
            if valLabels["keypointlabels"][0] in classNames:
                continue
            else:
                classNames.append(valLabels["polygonlabels"][0])
    if "label" in val:  
        for CountLabels,valLabels in enumerate(val["label"]):
            if len(classNames) == 0:
                classNames.append(valLabels["polygonlabels"][0])
            if valLabels["polygonlabels"][0] in classNames:
                continue
            else:
                classNames.append(valLabels["polygonlabels"][0])

print(classNames)
listOfAnnotations = []
cocoJSON = []        
for Count,val in enumerate(imgs_anns):
    getcount = [-1]*len(classNames)
    fileName = val["image"]
    filename_without_ext = os.path.splitext(fileName)[0]
    labelDestination = path_tostoreImagesAndAnnotations + filename_without_ext
    f = open(labelDestination+".txt","w+")
    
    if "label" in val:      
        for CountLabels,valLabels in enumerate(val["label"]):
            width = valLabels["original_width"]
            height = valLabels["original_height"]
            bboxArr = []
            for ind in range(0,len(getcount)):
                if valLabels["polygonlabels"][0] == classNames[ind]:
                    getcount[ind] += 1
                    catID = ind
            getPoints = np.array(valLabels["points"])
            getPoints[:,0] = (getPoints[:,0] * width/100)/width
            getPoints[:,1] = (getPoints[:,1] * height/100)/height

            bboxArr.append(catID)
            bboxArr.append(np.min(getPoints[:,0]) + (np.max(getPoints[:,0])-np.min(getPoints[:,0]))/2)
            bboxArr.append(np.min(getPoints[:,1]) + (np.max(getPoints[:,1])-np.min(getPoints[:,1]))/2)
            bboxArr.append(np.max(getPoints[:,0])-np.min(getPoints[:,0]))
            bboxArr.append(np.max(getPoints[:,1])-np.min(getPoints[:,1]))
            res = ""
            for i in bboxArr:
                res += str(i) + " "
            f.write(res)
            f.write("\n")
    
    if "kp-1" in val:
        for CountLabels,valLabels in enumerate(val["kp-1"]):
            bboxArr = []
            for ind in range(0,len(getcount)):
                if valLabels["keypointlabels"][0] == classNames[ind]:
                    getcount[ind] += 1
                    catID = ind
            xCorr = np.array(valLabels["x"])
            xCorrOtherPoint = xCorr+2
            xCorr = xCorr  - 2
            
            xCorr = (xCorr * width/100)/width
            xCorrOtherPoint = (xCorrOtherPoint * width/100)/width
            if (xCorrOtherPoint > 1) or (xCorr > 1):
                xCorrOtherPoint = 0.99
                
            
            yCorr = np.array(valLabels["y"])
            yCorrOtherPoint = yCorr+2
            yCorr = yCorr - 2
            
            yCorr = (yCorr * height/100)/height
            yCorrOtherPoint = (yCorrOtherPoint * height/100)/height
            if (yCorrOtherPoint > 1) or (yCorr > 1):
                yCorrOtherPoint = 0.99
                
            bboxArr.append(catID)
            bboxArr.append(xCorr + (xCorrOtherPoint-xCorr)/2)
            bboxArr.append(yCorr + (yCorrOtherPoint-yCorr)/2)
            bboxArr.append(xCorrOtherPoint-xCorr)
            bboxArr.append(yCorrOtherPoint-yCorr)
            res = ""
            for i in bboxArr:
                res += str(i) + " "
            f.write(res)
            f.write("\n")
            

    f.close()

In [ ]:
#arrays to store file names
imgs =[]
xmls =[]

#setup ratio (val ratio = rest of the files in origin dir after splitting into train and test)
train_ratio = 0.80  ##Argument 5
val_ratio = 0.20    ##Argument 6


#total count of imgs
totalImgCount = len(os.listdir(path_tostoreImagesAndAnnotations))/2

#soring files to corresponding arrays
for (dirname, dirs, files) in os.walk(path_tostoreImagesAndAnnotations):
    for filename in files:
        if filename.endswith('.txt'):
            xmls.append(filename)
        else:
            imgs.append(filename)
#counting range for cycles
countForTrain = int(len(imgs)*train_ratio)
countForVal = int(len(xmls)*val_ratio)
print("training images are : ",countForTrain)
print("Validation images are : ",countForVal)

In [ ]:
directory = "images"
path_forImages = os.path.join(OutputFolder, directory)
os.mkdir(path_forImages)
trainPath = path_forImages
path_forImages = path_forImages +"/"

directory = "train"
path_forImages_train = os.path.join(path_forImages, directory)
os.mkdir(path_forImages_train)

directory = "val"
path_forImages_val = os.path.join(path_forImages, directory)
os.mkdir(path_forImages_val)



directory = "labels"
path_forLabels = os.path.join(OutputFolder, directory)
os.mkdir(path_forLabels)
valPath = path_forLabels
path_forLabels = path_forLabels +"/"

directory = "train"
path_forLabels_train = os.path.join(path_forLabels, directory)
os.mkdir(path_forLabels_train)

directory = "val"
path_forLabels_val = os.path.join(path_forLabels, directory)
os.mkdir(path_forLabels_val)

In [ ]:
#cycle for train dir
for x in range(countForTrain):

    fileJpg = choice(imgs) # get name of random image from origin dir
    fileXml = fileJpg[:-4] +'.txt' # get name of corresponding annotation file

    #move both files into train dir
    #shutil.move(os.path.join(crsPath, fileJpg), os.path.join(trainimagePath, fileJpg))
    #shutil.move(os.path.join(crsPath, fileXml), os.path.join(trainlabelPath, fileXml))
    shutil.copy(os.path.join(path_tostoreImagesAndAnnotations, fileJpg), os.path.join(path_forImages_train, fileJpg))
    shutil.copy(os.path.join(path_tostoreImagesAndAnnotations, fileXml), os.path.join(path_forLabels_train, fileXml))


    #remove files from arrays
    imgs.remove(fileJpg)
    xmls.remove(fileXml)

#cycle for test dir   
for x in range(countForVal):

    fileJpg = choice(imgs) # get name of random image from origin dir
    fileXml = fileJpg[:-4] +'.txt' # get name of corresponding annotation file

    #move both files into train dir
    #shutil.move(os.path.join(crsPath, fileJpg), os.path.join(valimagePath, fileJpg))
    #shutil.move(os.path.join(crsPath, fileXml), os.path.join(vallabelPath, fileXml))
    shutil.copy(os.path.join(path_tostoreImagesAndAnnotations, fileJpg), os.path.join(path_forImages_val, fileJpg))
    shutil.copy(os.path.join(path_tostoreImagesAndAnnotations, fileXml), os.path.join(path_forLabels_val, fileXml))
    
    #remove files from arrays
    imgs.remove(fileJpg)
    xmls.remove(fileXml)
shutil.rmtree(path_tostoreImagesAndAnnotations, ignore_errors=True)